In [1]:
import math

import pretty_midi as midi
import numpy as np

For machine learning applications we are interested in preprocessing MIDI files of different orchestra works into a simplified, standardized format:

* Quantize notes
* Transpose all notes (octaves) within a fixed range
* Convert parts to one time signature
* Remove parts which are too sparse
* Reduce all parts to a fixed number

Through different techniques we want to augment the data:

* Transpose score to all keys (does not happen in this part)
* Through part reduction we deal with unused material, we use this material to generate new score material, changing the original score but keeping the "style" of the composition

In [2]:
# Input and output files
PATH_IN = './files/midi/symphony.mid'
PATH_OUT = './files/midi/symphony-converted.mid'

# Transpose all notes between this range (MIDI)
INTERVAL_LOW = 32
INTERVAL_HIGH = 72

# Defaults for every part and note
DEFAULT_BPM = 120
DEFAULT_INSTRUMENT = 'Acoustic Grand Piano'
DEFAULT_TIME_SIGNATURE = (3, 4)

# How many parts our output will contain
VOICE_NUM = 5

# How should the parts be distributed in %
VOICE_DISTRIBUTION = [0.1, 0.2, 0.3, 0.2, 0.2]

# Filter options
VALID_TIME_SIGNATURES = [DEFAULT_TIME_SIGNATURE, (6, 8)]
SCORE_PART_RATIO = 0.06

In [3]:
# Do some health checks before we start
test = 1.0 - np.sum(VOICE_DISTRIBUTION)
if test > 0.001 or test < 0:
    print('Warning: VOICE_DISTRIBUTION sum is not 1.0!')
    
if len(VOICE_DISTRIBUTION) != VOICE_NUM:
    print('Warning: length of VOICE_DISTRIBUTION is not equals VOICE_NUM!')
    
if INTERVAL_HIGH - INTERVAL_LOW < 12:
    print('Warning: Interval range is smaller than an octave!')

In [4]:
def get_end_time(score, bpm, time_signature):
    """Gets the normalized duration of a score."""
    
    # Get the score end time in seconds
    end_time = math.ceil(score.get_end_time() * 10) / 10

    # Calculate how long a measure is in seconds
    beat_time = midi.qpm_to_bpm(60 / bpm, time_signature[0], time_signature[1])
    measure_time = beat_time * time_signature[0]

    # Normalize the end time to a well formed measure
    end_time = end_time + (end_time % measure_time)
    
    return end_time

def copy_note(note, offset=0):
    """Safely make a new note instance."""
    
    return midi.Note(pitch=note.pitch,
                     start=note.start + offset,
                     end=note.end + offset,
                     velocity=note.velocity)

In [5]:
# Read MIDi file and clean up
score = midi.PrettyMIDI(PATH_IN)
score.remove_invalid_notes()
print('Loaded "{}".'.format(PATH_IN))

Loaded "./files/midi/symphony.mid".


After import of the MIDI file we filter out uneeded time signatures.

In [6]:
def filter_time_signatures(score, valid_time_signatures, bpm, time_signature):
    """Filters notes by time signature."""
    
    original_end_time = get_end_time(score, bpm, time_signature)
    
    # Detect times with correct time signatures
    valid_times = []
    valid_time = []

    for signature in score.time_signature_changes:
        is_valid_signature = False

        for valid_signature in valid_time_signatures:
            if (signature.numerator == valid_signature[0] and
                signature.denominator == valid_signature[1]):
                is_valid_signature = True
                print('Found {}.'.format(signature))

        if is_valid_signature:
            if len(valid_time) == 1:
                # Ignore this valid signature since we already have one.
                continue

            if len(valid_time) == 2:
                # This is already full, save it!
                valid_times.append(valid_time)

            # Keep the start time of this valid time period
            valid_time = [signature.time]

        else:
            # This is the end of a valid period
            if len(valid_time) == 1:
                valid_times.append([valid_time[0], signature.time])
                valid_time = []

    if len(valid_time) == 1:
        valid_times.append([valid_time[0], original_end_time])

    print('Total {} valid time frame(s).'.format(len(valid_times)))
    
    # Create a new score with only valid time signatures
    new_score = midi.PrettyMIDI(initial_tempo=bpm)

    for instrument in score.instruments:
        new_instrument = midi.Instrument(program=instrument.program)
        pitches = []
        for note in instrument.notes:
            offset = 0
            for valid_time in valid_times:
                offset += valid_time[0]
                if not (note.end <= valid_time[0] or note.start >= valid_time[1]):
                    new_instrument.notes.append(copy_note(note, -offset))
        new_score.instruments.append(new_instrument)
        
    end_time = get_end_time(new_score, bpm, time_signature)
    print('New score has a length of {0:.4} seconds (original was {1:.4} seconds).'.format(
        end_time, original_end_time))
    
    if end_time / original_end_time < 0.1:
        print('Warning: A large part of the original score was removed!')
        
    return new_score

In [7]:
# Remove invalid time signatures
temp_score = filter_time_signatures(score, VALID_TIME_SIGNATURES, DEFAULT_BPM, DEFAULT_TIME_SIGNATURE)

Found 3/4 at 0.00 seconds.
Total 1 valid time frame(s).
New score has a length of 42.9 seconds (original was 42.9 seconds).


Now we check every single part of the score for its amount of notes in comparison to the whole score. If the ratio is very low (almost only empty staves) we ignore this part for later use (`SCORE_PART_RATIO`).

In [8]:
def remove_sparse_parts(score, ratio):
    """Remove parts which are too sparse."""

    original_instruments_count = len(score.instruments)
    original_notes_count = 0
    for instrument in score.instruments:
        original_notes_count += len(instrument.notes)
        
    removed_instruments = []

    for instrument_index, instrument in enumerate(score.instruments):
        instrument_notes_count = len(instrument.notes)
        if instrument_notes_count == 0:
            instrument_score_ratio = 0
        else:
            instrument_score_ratio = (instrument_notes_count / original_notes_count)

        print('Part #{0} with a note ratio score of {1:.2%}'.format(
            instrument_index + 1,
            instrument_score_ratio))

        if instrument_score_ratio < ratio:
            removed_instruments.append(instrument_index)
    
    for instrument_index in reversed(removed_instruments):
        del score.instruments[instrument_index]

    print('Removed {} part(s), now {} given (original had {}).'.format(
        original_instruments_count - len(score.instruments),
        len(score.instruments),
        original_instruments_count))

In [9]:
# Remove sparse instruments
remove_sparse_parts(temp_score, SCORE_PART_RATIO)

Part #1 with a note ratio score of 11.90%
Part #2 with a note ratio score of 11.52%
Part #3 with a note ratio score of 10.63%
Part #4 with a note ratio score of 5.57%
Part #5 with a note ratio score of 0.89%
Part #6 with a note ratio score of 5.57%
Part #7 with a note ratio score of 1.52%
Part #8 with a note ratio score of 14.56%
Part #9 with a note ratio score of 6.71%
Part #10 with a note ratio score of 11.65%
Part #11 with a note ratio score of 13.42%
Part #12 with a note ratio score of 6.08%
Removed 4 part(s), now 8 given (original had 12).


To reduce all given parts to a smaller number we first analyze the ambitus of every part (not the instrument since this data might not always be correct) to then calculate a score (0 - 100%) of how likely this ambitus will fit into our new range based on `VOICE_DISTRIBUTION`.

In [10]:
def identify_ambitus_groups(score, voice_num, voice_distribution):
    """Finds out which parts of the score belong to which ambitus group."""

    print('Identify ambitus groups for all score parts ..')

    # 1. Analyze ambitus for every part
    instrument_intervals = []
    for instrument_index, instrument in enumerate(score.instruments):
        pitches = []
        for note in instrument.notes:
            pitches.append(note.pitch)
            
        instrument_intervals.append([instrument_index,
                                     min(pitches),
                                     max(pitches)])

    # 2. Identify minimum and maximum ambitus over all parts
    instrument_intervals = np.array(instrument_intervals)
    interval_min = np.min(instrument_intervals[:, 1])
    interval_max = np.max(instrument_intervals[:, 2])
    print('Score ambitus is {} - {} (min - max)!'.format(interval_min,
                                                         interval_max))

    # 3. Calculate closeness for every part to our groups
    scores = []
    interval_total = interval_max - interval_min
    interval_slice = math.ceil(interval_total / voice_num)

    range_min = interval_min - 1
    for voice_index in range(0, voice_num):
        range_min = interval_min + (interval_slice * voice_index) + 1
        range_max = interval_min + (
            interval_slice * voice_index) + interval_slice
        for interval in instrument_intervals:
            instrument_index, instrument_min, instrument_max = interval
            closeness = abs(range_min - instrument_min) + abs(range_max - instrument_max)
            closeness = 1 - (closeness / (interval_total * 2))
            scores.append([instrument_index,
                           voice_index,
                           closeness])

    # 4. Group parts based on closeness
    instrument_groups = []
    groups_count = [0 for i in range(0, voice_num)]

    for interval in instrument_intervals:
        # Filter all closeness scores belonging to this part ...
        instrument_index = interval[0]
        instrument_scores = list(filter(lambda i: i[0] == instrument_index, scores))
        # ... sort them ...
        instrument_scores = sorted(instrument_scores, key=lambda i: i[2], reverse=True)
        # ... and take the group with the best score.
        group_index = instrument_scores[0][1]
        find_direction = True
        while (groups_count[group_index] / len(instrument_intervals)
               > voice_distribution[group_index]):
            # Change group index when first choice was too full
            if group_index == voice_num - 1:
                find_direction = False
            elif group_index == 0:
                find_direction = True
            group_index += 1 if find_direction else -1

        instrument_groups.append(group_index)
        groups_count[group_index] += 1

    groups_count = np.array(groups_count)

    # 5. Fill up empty spaces
    while len(np.where(groups_count == 0)[0]) > 0:
        empty_group_index = np.where(groups_count == 0)[0][0]
        full_group_index = np.argmax(groups_count)
        instrument_index = np.argwhere(instrument_groups == full_group_index).flatten()[0]
        instrument_groups[instrument_index] = empty_group_index
        groups_count[empty_group_index] += 1
        groups_count[full_group_index] -= 1
        print('Empty group {} detected, fill it up with part {}!'.format(
            empty_group_index, instrument_index))

    print('Parts in groups:', instrument_groups)

    return np.array(instrument_groups)

In [11]:
# Identify ambitus group for every instrument
groups = identify_ambitus_groups(temp_score, VOICE_NUM, VOICE_DISTRIBUTION)

Identify ambitus groups for all score parts ..
Score ambitus is 28 - 88 (min - max)!
Parts in groups: [3, 3, 4, 4, 2, 2, 1, 0]


We transpose the notes into a defined range.

In [12]:
def transpose(score, interval_min, interval_max):
    """Transpose all notes within a given interval."""
    
    for instrument in score.instruments:
        for note in instrument.notes:
            normalized_pitch = note.pitch % 12
            if note.pitch > interval_max:
                normalized_interval = interval_max % 12
                new_pitch = interval_max - normalized_interval - (12 - normalized_pitch)
            elif note.pitch < interval_min:
                normalized_interval = interval_min % 12
                new_pitch = interval_min + normalized_interval + normalized_pitch
            else:
                new_pitch = note.pitch
                
            note.pitch = new_pitch

In [13]:
# Transpose within an interval
transpose(temp_score, INTERVAL_LOW, INTERVAL_HIGH)

Since we reduced the number of parts to x groups (`VOICE_NUM`) we will need to ignore original parts in the final score. To avoid this (and to augment our data), we don't throw away any unused parts but generate multiple scores in all different part combinations.

The combinations are calculated by first translating the part groups into a tree structure to then traverse the tree from bottom up, finding all possible combinations.

In [14]:
def create_combination_tree(options, group_index):
    """Convert all possible combinations into a tree data structure."""
    
    if len(options) - 1 < group_index:
        return None
    
    combinations = []
    
    for option in options[group_index]:
        combinations.append(option)
        results = create_combination_tree(options, group_index + 1)
        if results:
            combinations.append(results)
            
    return combinations


def traverse_combination_tree(tree, single_combination = [], result = [], depth = 0):
    """Traverse a tree to find all possible combinations."""
    
    if not hasattr(tree, '__len__'):
        return single_combination
    
    if depth == 0:
        result = []
        
    if len(tree) == 1:
        result.append(traverse_combination_tree(tree[0],
                                                single_combination + [tree[0]],
                                                result,
                                                depth + 1))
        
    for i in range(0, len(tree) - 1, 2):
        sub_tree = tree[i + 1]
        if not hasattr(sub_tree, '__len__'):
            for n in tree:
                result.append(
                    traverse_combination_tree(n,
                                              single_combination + [n],
                                              result,
                                              depth + 1))
        else:
            traverse_combination_tree(sub_tree,
                                      single_combination + [tree[i]],
                                      result,
                                      depth + 1)
            
    return result

In [15]:
# Check which parts we can combine
combination_options = []
for group_index in range(0, VOICE_NUM):
    options = np.argwhere(groups == group_index).flatten()
    combination_options.append(options)
    print('Parts {} in group {} (size = {}).'.format(
        options, group_index, len(options)))

# Build a tree to traverse to find all combinations
tree = create_combination_tree(combination_options, 0)
combinations = traverse_combination_tree(tree, single_combination=[])

print('Found {} possible combinations.'.format(len(combinations)))

Parts [7] in group 0 (size = 1).
Parts [6] in group 1 (size = 1).
Parts [4 5] in group 2 (size = 2).
Parts [0 1] in group 3 (size = 2).
Parts [2 3] in group 4 (size = 2).
Found 8 possible combinations.


Finally we can generate a new score based on our temporary score with cleaned original parts and the given possible combinations. All combinations are simply just added up, the score grows in this way and yields more data.

In [16]:
# Prepare a new score with empty parts for every voice
new_score = midi.PrettyMIDI(initial_tempo=DEFAULT_BPM)
temp_end_time = get_end_time(temp_score, DEFAULT_BPM, DEFAULT_TIME_SIGNATURE)

new_score.time_signature_changes = [midi.TimeSignature(
    numerator=DEFAULT_TIME_SIGNATURE[0],
    denominator=DEFAULT_TIME_SIGNATURE[1],
    time=0.0)]

for i in range(0, VOICE_NUM):
    program = midi.instrument_name_to_program(DEFAULT_INSTRUMENT)
    new_instrument = midi.Instrument(program=program)
    new_score.instruments.append(new_instrument)

# Add parts in all possible combinations
for combination_index, combination in enumerate(combinations):
    offset = combination_index * temp_end_time
    for instrument_index, temp_instrument_index in enumerate(
            reversed(combination)):
        for note in temp_score.instruments[temp_instrument_index].notes:
            new_score.instruments[instrument_index].notes.append(
                copy_note(note, offset))
            
    print('Generated combination {} #{}.'.format(
        combination, combination_index + 1))
    
# Done!
new_end_time = get_end_time(new_score, DEFAULT_BPM, DEFAULT_TIME_SIGNATURE)
print('Generated score with duration {0:.4} seconds. '
      'Data augmentation of {1:.0%}!'.format(
          new_end_time,
          ((new_end_time / temp_end_time) - 1)))

Generated combination [7, 6, 4, 0, 2] #1.
Generated combination [7, 6, 4, 0, 3] #2.
Generated combination [7, 6, 4, 1, 2] #3.
Generated combination [7, 6, 4, 1, 3] #4.
Generated combination [7, 6, 5, 0, 2] #5.
Generated combination [7, 6, 5, 0, 3] #6.
Generated combination [7, 6, 5, 1, 2] #7.
Generated combination [7, 6, 5, 1, 3] #8.
Generated score with duration 342.0 seconds. Data augmentation of 697%!


In [17]:
# Write result to MIDI file
new_score.write(PATH_OUT)